In [5]:
import pandas as pd
import numpy as np
import time
import sys

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression  
from sklearn import svm

module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/'
if module_path not in sys.path:
    sys.path.append(module_path)

from cohort_loader_new_version0229 import *
import eicu_year_process

module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation/'
if module_path not in sys.path:
    sys.path.append(module_path)

import get_hospital_eicu

import split
from imp import reload
reload(split)

import Multiclass_evaluation
reload(Multiclass_evaluation)

<module 'Multiclass_evaluation' from 'c:\\Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation\\Multiclass_evaluation.py'>

In [6]:
mimic_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/mimic_analysis(new_version0308).csv.gz'
eicu_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/eicu_analysis(new_version0308).csv.gz'

mimic = pd.read_csv(mimic_path, compression = 'gzip')
mimic = mimic.drop('Shock_next_12h', axis = 1)

eicu = pd.read_csv(eicu_path, compression = 'gzip')
eicu = eicu.drop('Shock_next_12h', axis = 1)

dataset_mimic = mimic[~(mimic['gender']==2)].reset_index(drop=True)

dataset_mimic.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset_mimic.fillna(0, inplace=True) 
dataset_mimic = dataset_mimic[~(dataset_mimic['Case']=='event')]
dataset_mimic = dataset_mimic[~((dataset_mimic['Case']==4)&(dataset_mimic['Annotation']=='no_circ'))]

eventset = mimic[(mimic['Case']=='event')].reset_index(drop=True)
dataset_mimic['Case'] = pd.to_numeric(dataset_mimic['Case'], errors='coerce')

mimic_train, mimic_valid = data_split(dataset_mimic, 9040, 0.9, Threshold=0.05, n_trial=1, mode = 'mimic')

eicu_t = eicu_year_process.matching_patient(eicu)
dataset_eicu = eicu_t[~(eicu_t['gender']==2)].reset_index(drop=True)
dataset_eicu.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset_eicu.fillna(0, inplace=True) 

eicu_test = dataset_eicu[~(dataset_eicu['Case']=='event')]
event_eicu = dataset_eicu[(dataset_eicu['Case']=='event')]
eicu_test = eicu_test[~((eicu_test['Case']==4)&(eicu_test['Annotation']=='no_circ'))]
eicu_test['Case'] = pd.to_numeric(eicu_test['Case'], errors='coerce')
hosp_id, eicu_test = get_hospital_eicu.hospital(eicu_test)

eicu_type = get_hospital_eicu.eicu_subgroup(eicu_test)
unitadmitsource, unittype, unitstaytype = get_hospital_eicu.make_eicu_dataset(eicu_type)

========== 데이터셋 분할 정보 ==========
데이터셋 비율: 학습 = 0.90, 테스트 = 0.10
학습셋 클래스 비율: Case
1.0    473941
2.0      5128
3.0     13766
4.0     17961
Name: count, dtype: int64
테스트셋 클래스 비율: Case
1.0    53103
2.0      512
3.0     1191
4.0     1383
Name: count, dtype: int64
--------------------------------------
========== 클래스 비율 ==========
학습셋 클래스 비율: 0.93:0.01:0.03:0.04
테스트셋 클래스 비율: 0.95:0.01:0.02:0.02
--------------------------------------
========== 환자 및 체류 정보 ==========
학습셋 환자 수: 2907
테스트셋 환자 수: 329
학습셋 체류 수: 2961
테스트셋 체류 수: 330
--------------------------------------
========== 실험 설정 ==========
분할 시드: 9040
학습 비율: 0.9
임계값: 0.05
--------------------------------------
========== 실행 결과 ==========
총 소요 시간(초): 0.39
시도한 시행 횟수: 0


In [7]:
reload(split)
X_train, y_train, _ = split.split_X_Y(mimic_train, mode = 'mimic')
X_valid, y_valid, valid_output = split.split_X_Y(mimic_valid, mode = 'mimic')

In [8]:
valid_event = eventset[eventset.stay_id.isin(mimic_valid.stay_id.unique())]

In [4]:
# from sklearn.preprocessing import MinMaxScaler

# scaler = MinMaxScaler()
# X_num_scaled = scaler.fit_transform(X_train_case1_2)
# X_train_case1_2_scaled = pd.DataFrame(X_num_scaled,columns = X_train_case1_2.columns)

# X_valid_case1_2_scaled = scaler.transform(X_valid_case1_2)
# X_valid_case1_2_scaled = pd.DataFrame(X_valid_case1_2_scaled,columns = X_valid_case1_2.columns)

# X_test_case1_2_scaled = scaler.transform(X_test_case1_2)
# X_test_case1_2_scaled = pd.DataFrame(X_test_case1_2_scaled,columns = X_test_case1_2.columns)

In [4]:
reload(Multiclass_evaluation) #no tuning 2h
models, result = Multiclass_evaluation.create_analysis(eventset, X_train, y_train, X_valid, valid_output, mode = 'base')

|MIMIC-Validation|====================================================


,Model,Accuracy,Case 1 AUROC,Case 1 AUPRC,Case 2 AUROC,Case 2 AUPRC,Case 3 AUROC,Case 3 AUPRC,Case 4 AUROC,Case 4 AUPRC,Case 1 Score,Case 2 Score,Case 3 Score,Case 4 Score
0,lgbm,0.983822,0.9211,0.9955,0.5581,0.3353,0.9000,0.8355,0.9373,0.8622,0.9950,0.4141,0.8322,0.8604
1,rf,0.984374,0.9223,0.9955,0.5466,0.3510,0.9182,0.8484,0.9297,0.8624,0.9952,0.4054,0.8465,0.8608
2,dt,0.949972,0.9160,0.9951,0.5886,0.1360,0.8951,0.8279,0.9358,0.8488,0.9772,0.1985,0.8243,0.8463
3,lr,0.986581,0.9237,0.9956,0.5426,0.2916,0.9602,0.9068,0.9448,0.9094,0.9952,0.3073,0.9057,0.9078


----------------------------------------------------------------------


In [9]:
reload(Multiclass_evaluation) #no tuning 3h
models, result = Multiclass_evaluation.create_analysis(eventset, X_train, y_train, X_valid, valid_output, mode = 'base')

|MIMIC-Validation|====================================================


,Model,Accuracy,Case 1 AUROC,Case 1 AUPRC,Case 2 AUROC,Case 2 AUPRC,Case 3 AUROC,Case 3 AUPRC,Case 4 AUROC,Case 4 AUPRC,Case 1 Score,Case 2 Score,Case 3 Score,Case 4 Score
0,lgbm,0.982701,0.9224,0.9955,0.5611,0.3443,0.8668,0.7963,0.9399,0.8476,0.9951,0.4155,0.7894,0.8444
1,rf,0.982399,0.9174,0.9952,0.5406,0.2838,0.8779,0.7998,0.9311,0.8466,0.9948,0.3500,0.7954,0.8444
2,dt,0.947036,0.9160,0.9951,0.5886,0.1360,0.8345,0.7410,0.9252,0.8068,0.9772,0.1985,0.7318,0.8016
3,lr,0.985905,0.9237,0.9956,0.5426,0.2943,0.9572,0.8887,0.9337,0.8985,0.9952,0.3077,0.8868,0.8960


----------------------------------------------------------------------


In [10]:
reload(Multiclass_evaluation) #no tuning
models, result = Multiclass_evaluation.create_analysis(valid_event, X_train, y_train, X_valid, valid_output, mode = 'base')

|MIMIC-Validation|====================================================


,Model,Accuracy,Case 1 AUROC,Case 1 AUPRC,Case 2 AUROC,Case 2 AUPRC,Case 3 AUROC,Case 3 AUPRC,Case 4 AUROC,Case 4 AUPRC,Case 1 Score,Case 2 Score,Case 3 Score,Case 4 Score
0,lgbm,0.987602,0.9242,0.9957,0.5591,0.3335,0.9694,0.9260,0.9525,0.9256,0.9952,0.3560,0.9251,0.9242
1,rf,0.985518,0.9168,0.9953,0.5456,0.3265,0.9432,0.8878,0.9335,0.8850,0.9949,0.4150,0.8866,0.8833
2,dt,0.951674,0.9149,0.9951,0.5886,0.1360,0.9245,0.8702,0.9440,0.8764,0.9772,0.1985,0.8682,0.8749
3,lr,0.987709,0.9232,0.9956,0.5473,0.2747,0.9681,0.9347,0.9647,0.9352,0.9951,0.3323,0.9340,0.9344


----------------------------------------------------------------------


In [48]:
reload(Multiclass_evaluation)

X_train, y_train, output = split.split_X_Y(mimic_train[mimic_train['stay_id'].isin(eventset.stay_id.unique())], mode = 'mimic')
X_valid, y_valid, valid_output = split.split_X_Y(mimic_valid, mode = 'mimic')

models, result = Multiclass_evaluation.create_analysis(eventset, X_train, y_train, X_valid, valid_output, mode = 'base')

|MIMIC-Validation|====================================================


,Model,Accuracy,Case 1 AUROC,Case 1 AUPRC,Case 2 AUROC,Case 2 AUPRC,Case 3 AUROC,Case 3 AUPRC,Case 4 AUROC,Case 4 AUPRC,Case 1 Score,Case 2 Score,Case 3 Score,Case 4 Score
0,lgbm,0.985696,0.9304,0.9960,0.5968,0.2706,0.9589,0.9236,0.9629,0.9252,0.9943,0.3785,0.9227,0.9243
1,rf,0.973922,0.9299,0.9960,0.6286,0.2220,0.9647,0.8828,0.9422,0.8812,0.9883,0.4247,0.8790,0.8799
2,dt,0.820588,0.8633,0.9919,0.6319,0.2309,0.9146,0.8622,0.9469,0.8720,0.8997,0.1042,0.8594,0.8700
3,lr,0.983202,0.9325,0.9962,0.6177,0.2476,0.9653,0.9311,0.9638,0.9318,0.9928,0.4229,0.9303,0.9310


----------------------------------------------------------------------


In [51]:
result.to_excel(excel_writer='internal_validation_baseline_method.xlsx')

In [46]:
reload(Multiclass_evaluation)

X_train, y_train, output = split.split_X_Y(mimic_train[mimic_train['stay_id'].isin(eventset.stay_id.unique())], mode = 'mimic')
X_valid, y_valid, valid_output = split.split_X_Y(mimic_valid[mimic_valid['stay_id'].isin(eventset.stay_id.unique())], mode = 'mimic')

models, result = Multiclass_evaluation.create_analysis(eventset, X_train, y_train, X_valid, valid_output, mode = 'base')

|MIMIC-Validation|====================================================


,Model,Accuracy,Case 1 AUROC,Case 1 AUPRC,Case 2 AUROC,Case 2 AUPRC,Case 3 AUROC,Case 3 AUPRC,Case 4 AUROC,Case 4 AUPRC,Case 1 Score,Case 2 Score,Case 3 Score,Case 4 Score
0,lgbm,0.923815,0.9308,0.9620,0.5976,0.5593,0.9530,0.9289,0.9564,0.9300,0.9597,0.4390,0.9227,0.9243
1,rf,0.902769,0.9194,0.9574,0.6266,0.4211,0.9549,0.9102,0.9346,0.9027,0.9454,0.6479,0.9042,0.8938
2,dt,0.810954,0.8633,0.9344,0.6574,0.3331,0.9063,0.8733,0.9335,0.8788,0.8750,0.5226,0.8594,0.8700
3,lr,0.923446,0.9303,0.9620,0.6172,0.4838,0.9595,0.9355,0.9584,0.9366,0.9575,0.5415,0.9303,0.9310


----------------------------------------------------------------------


In [34]:
#optuna
import optuna
from optuna.samplers import TPESampler
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report

module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation/'
if module_path not in sys.path:
    sys.path.append(module_path)
    
import classifier_ML
reload(classifier_ML)

sampler = TPESampler(seed=10)

X_train, y_train, output = split.split_X_Y(mimic_train[mimic_train['stay_id'].isin(eventset.stay_id.unique())], mode = 'mimic')
X_valid, y_valid, valid_output = split.split_X_Y(mimic_valid, mode = 'mimic')

def objective(trial):

    param = {
        'objective': 'multiclass',
        'verbose': -1,
        'metric': 'multi_logloss', 
        "num_class": 4,
        'max_depth': trial.suggest_int('max_depth',10, 100),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 9, 11),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'subsample': trial.suggest_loguniform('subsample', 0.4, 1),

    }

    model = LGBMClassifier(**param, early_stopping_rounds=25, extra_trees = True, random_state =42)
    lgb_model = model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])
    valid_preds = lgb_model.predict(X_valid)
    valid_output['prediction_label'] = valid_preds
    valid_output['prediction_prob'] = lgb_model.predict_proba(X_valid)[:, 1]
    
    result = Multiclass_evaluation.MULT_evaluation(valid_output, eventset, 'lgbm', 'mimic')
    
    return result['Case 2 Score'].values[0]
        
study_lgb = optuna.create_study(direction='maximize', sampler=sampler)
study_lgb.optimize(objective, n_trials=30)
print('LightGBM Best parameters:', study_lgb.best_params)

[I 2024-03-03 20:53:24,979] A new study created in memory with name: no-name-2f976033-30b2-452a-b171-af1c3c42bd82
[I 2024-03-03 20:53:25,979] Trial 0 finished with value: 0.0 and parameters: {'max_depth': 80, 'lambda_l1': 1.537331564587801e-08, 'lambda_l2': 0.005044685709888605, 'learning_rate': 0.0003110450506452811, 'n_estimators': 10, 'min_child_samples': 26, 'subsample': 0.479597741694721}. Best is trial 0 with value: 0.0.


ZeroDivisionError


[I 2024-03-03 20:53:26,971] Trial 1 finished with value: 0.0 and parameters: {'max_depth': 79, 'lambda_l1': 3.32657660618516e-07, 'lambda_l2': 6.238186113062485e-08, 'learning_rate': 0.0001294669479886693, 'n_estimators': 11, 'min_child_samples': 5, 'subsample': 0.6395607233867681}. Best is trial 0 with value: 0.0.


ZeroDivisionError


[I 2024-03-03 20:53:27,974] Trial 2 finished with value: 0.0 and parameters: {'max_depth': 83, 'lambda_l1': 0.003256376421394008, 'lambda_l2': 0.03131827670437561, 'learning_rate': 5.639705277794729e-07, 'n_estimators': 11, 'min_child_samples': 73, 'subsample': 0.6575974218219474}. Best is trial 0 with value: 0.0.


...

ZeroDivisionError


[I 2024-03-03 20:53:29,980] Trial 4 finished with value: 0.0 and parameters: {'max_depth': 69, 'lambda_l1': 0.0025665550309028774, 'lambda_l2': 0.1766073489911767, 'learning_rate': 1.3485992516968212e-05, 'n_estimators': 11, 'min_child_samples': 35, 'subsample': 0.4345676420333751}. Best is trial 0 with value: 0.0.


ZeroDivisionError


[W 2024-03-03 20:53:30,192] Trial 5 failed with parameters: {'max_depth': 37, 'lambda_l1': 1.0613515458050516e-07, 'lambda_l2': 0.2871652845130526, 'learning_rate': 1.9115159066086765e-08, 'n_estimators': 10, 'min_child_samples': 57, 'subsample': 0.8473972796749784} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\DAHS\anaconda3\envs\DL\lib\site-packages\optuna\study\_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\DAHS\AppData\Local\Temp\ipykernel_8992\1905087887.py", line 37, in objective
    lgb_model = model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)])
  File "c:\Users\DAHS\anaconda3\envs\DL\lib\site-packages\lightgbm\sklearn.py", line 1187, in fit
    super().fit(
  File "c:\Users\DAHS\anaconda3\envs\DL\lib\site-packages\lightgbm\sklearn.py", line 885, in fit
    self._Booster = train(
  File "c:\Users\DAHS\anaconda3\envs\DL\lib\site-packages\lightgbm\engine.py", line 255, in t

KeyboardInterrupt: 

In [12]:
reload(Multiclass_evaluation)

# lgbm_tune = LGBMClassifier(max_depth = 82, lambda_l1 = 8.712893661203505e-06, lambda_l2 = 6.936859625682649e-06, 
#                            learning_rate = 0.00035344989482578203, 
#                            n_estimators = 10, min_child_samples = 6, subsample = 0.6049342836247393,
#                            random_state = 42, extra_trees = True, verbose=-1, class_weight='balanced')


lgbm_tune = LGBMClassifier(max_depth = 82, lambda_l1 = 8.712893661203505e-06, lambda_l2 = 6.936859625682649e-06, 
                       
                           n_estimators = 10, min_child_samples = 6, subsample = 0.6049342836247393,
                           random_state = 42, extra_trees = True, verbose=-1)


lgbm_tune.fit(X_train, y_train)

valid_preds = lgbm_tune.predict(X_valid)
valid_output['prediction_label'] = valid_preds
valid_output['prediction_prob'] = lgbm_tune.predict_proba(X_valid)[:, 1]
    
validation_tune = Multiclass_evaluation.MULT_evaluation(valid_output, eventset, 'lgbm', 'mimic')
print('Tuned evaluation')
display(validation_tune)

result_external = Multiclass_evaluation.external_evaluation(unitadmitsource, unittype, unitstaytype, lgbm_tune, 'LightGBM', event_eicu)
result_external.to_excel(excel_writer='external_validation_baseline_method.xlsx')

Tuned evaluation


,Model,Accuracy,Case 1 AUROC,Case 1 AUPRC,Case 2 AUROC,Case 2 AUPRC,Case 3 AUROC,Case 3 AUPRC,Case 4 AUROC,Case 4 AUPRC,Case 1 Score,Case 2 Score,Case 3 Score,Case 4 Score
0,lgbm,0.984414,0.9212,0.9955,0.5529,0.2528,0.9303,0.8678,0.9329,0.8753,0.9948,0.3389,0.8663,0.8738


[Starting eICU-Test]
eICU-UnitAdmitSource...
eICU-UnitType...
eICU-UnitStayType...
ZeroDivisionError


Model  Accuracy  Case 1 AUROC  \
Subpopulation   eICU Type                                                
UnitAdmitSource Floor                 LightGBM  0.985553        0.8931   
                Acute Care/Floor      LightGBM  0.983135        0.8497   
                Emergency Department  LightGBM  0.987303        0.8986   
                Direct Admit          LightGBM  0.990824        0.8938   
                Operating Room        LightGBM  0.989656        0.8826   
                Step-Down Unit (SDU)  LightGBM  0.982190        0.8728   
                Other ICU             LightGBM  0.986040        0.8312   
                Recovery Room         LightGBM  0.993987        0.9430   
                Other Hospital        LightGBM  0.986726        0.8399   
                ICU                   LightGBM  0.990598        0.8335   
                Chest Pain Center     LightGBM  0.971106        0.7900   
                PACU                  LightGBM  0.990378        0.9606   
UnitType        Med-Surg ICU          LightGBM  0.986620        0.8863   
                MICU                  LightGBM  0.987112        0.8999   
                Cardiac ICU           LightGBM  0.985219        0.8732   
                Neuro ICU             LightGBM  0.993674        0.8953   
                CTICU                 LightGBM  0.992109        0.8744   
                CSICU                 LightGBM  0.980581        0.8790   
                CCU-CTICU             LightGBM  0.985101        0.8721   
                SICU                  LightGBM  0.990197        0.9240   
UnitStayType    admit                 LightGBM  0.988142        0.8894   
                transfer              LightGBM  0.984873        0.8889   
                readmit               LightGBM  0.986049        0.8930   
                stepdown/other        LightGBM  0.993886        0.5000   

                                      Case 1 AUPRC  Case 2 AUROC  \
Subpopulation   eICU Type                                          
UnitAdmitSource Floor                       0.9948        0.5467   
                Acute Care/Floor            0.9942        0.5274   
                Emergency Department        0.9957        0.5369   
                Direct Admit                0.9973        0.5386   
                Operating Room              0.9964        0.5157   
                Step-Down Unit (SDU)        0.9938        0.5191   
                Other ICU                   0.9959        0.5102   
                Recovery Room               0.9983        0.5232   
                Other Hospital              0.9948        0.5219   
                ICU                         0.9966        0.5494   
                Chest Pain Center           0.9863        0.5625   
                PACU                        0.9955        0.6034   
UnitType        Med-Surg ICU                0.9953        0.5379   
                MICU                        0.9960        0.5436   
                Cardiac ICU                 0.9945        0.5171   
                Neuro ICU                   0.9979        0.5277   
                CTICU                       0.9975        0.5079   
                CSICU                       0.9928        0.5345   
                CCU-CTICU                   0.9951        0.5260   
                SICU                        0.9966        0.5498   
UnitStayType    admit                       0.9958        0.5355   
                transfer                    0.9956        0.5331   
                readmit                     0.9955        0.5275   
                stepdown/other              0.9973        0.4997   

                                      Case 2 AUPRC Case 3 AUROC Case 3 AUPRC  \
Subpopulation   eICU Type                                                      
UnitAdmitSource Floor                       0.1746       0.9149       0.8957   
                Acute Care/Floor            0.0898       0.8844        0.863   
                Emergency D

In [44]:
eicu_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/eicu_analysis(new_version0229).csv.gz'

eicu = pd.read_csv(eicu_path, compression = 'gzip')
eicu = eicu.drop('Shock_next_12h', axis = 1)

eicu_t = eicu_year_process.matching_patient(eicu)
dataset_eicu = eicu_t[~(eicu_t['gender']==2)].reset_index(drop=True)
dataset_eicu.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset_eicu.fillna(0, inplace=True) 

eicu_test = dataset_eicu[~(dataset_eicu['Case']=='event')]
event_eicu = dataset_eicu[(dataset_eicu['Case']=='event')]
eicu_test = eicu_test[~((eicu_test['Case']==4)&(eicu_test['Annotation']=='no_circ'))]
eicu_test['Case'] = pd.to_numeric(eicu_test['Case'], errors='coerce')
hosp_id, eicu_test = get_hospital_eicu.hospital(eicu_test[eicu_test.patientunitstayid.isin(event_eicu.patientunitstayid.unique())])

eicu_type = get_hospital_eicu.eicu_subgroup(eicu_test)
unitadmitsource_event, unittype_event, unitstaytype_event = get_hospital_eicu.make_eicu_dataset(eicu_type)

In [45]:
result_only_event = Multiclass_evaluation.external_evaluation(unitadmitsource_event, unittype_event, unitstaytype_event, lgbm_tune, 'LightGBM', event_eicu)

[Starting eICU-Test]
eICU-UnitAdmitSource...
eICU-UnitType...
eICU-UnitStayType...
ZeroDivisionError


Model  Accuracy  Case 1 AUROC  \
Subpopulation   eICU Type                                                
UnitAdmitSource Emergency Department  LightGBM  0.921560        0.9196   
                Other ICU             LightGBM  0.925392        0.8245   
                Direct Admit          LightGBM  0.925673        0.9178   
                Operating Room        LightGBM  0.907878        0.9278   
                Floor                 LightGBM  0.919825        0.9143   
                Other Hospital        LightGBM  0.933821        0.9253   
                Recovery Room         LightGBM  0.926647        0.9593   
                Acute Care/Floor      LightGBM  0.904705        0.8846   
                Chest Pain Center     LightGBM  0.814516        0.8162   
                Step-Down Unit (SDU)  LightGBM  0.925270        0.8908   
                PACU                  LightGBM  0.968064        0.9738   
                ICU                   LightGBM  0.959839        0.8554   
UnitType        Med-Surg ICU          LightGBM  0.925427        0.9168   
                Neuro ICU             LightGBM  0.929491        0.9064   
                Cardiac ICU           LightGBM  0.920655        0.8830   
                SICU                  LightGBM  0.928892        0.9441   
                CTICU                 LightGBM  0.915864        0.8819   
                CCU-CTICU             LightGBM  0.902277        0.8963   
                MICU                  LightGBM  0.926331        0.9207   
                CSICU                 LightGBM  0.914609        0.9210   
UnitStayType    transfer              LightGBM  0.936761        0.9157   
                admit                 LightGBM  0.920563        0.9168   
                readmit               LightGBM  0.916321        0.9180   
                stepdown/other        LightGBM  0.907216        0.5000   

                                      Case 1 AUPRC  Case 2 AUROC  \
Subpopulation   eICU Type                                          
UnitAdmitSource Emergency Department        0.9598        0.5785   
                Other ICU                   0.9652        0.5578   
                Direct Admit                0.9677        0.6108   
                Operating Room              0.9387        0.5407   
                Floor                       0.9599        0.5749   
                Other Hospital              0.9715        0.5737   
                Recovery Room               0.9524        0.5980   
                Acute Care/Floor            0.9590        0.5392   
                Chest Pain Center           0.8971        0.5081   
                Step-Down Unit (SDU)        0.9612        0.5510   
                PACU                        0.9529        0.7370   
                ICU                         0.9838        0.5890   
UnitType        Med-Surg ICU                0.9609        0.5621   
                Neuro ICU                   0.9615        0.5732   
                Cardiac ICU                 0.9594        0.5527   
                SICU                        0.9620        0.5933   
                CTICU                       0.9634        0.5845   
                CCU-CTICU                   0.9552        0.5824   
                MICU                        0.9657        0.5755   
                CSICU                       0.9591        0.5885   
UnitStayType    transfer                    0.9684        0.5904   
                admit                       0.9604        0.5730   
                readmit                     0.9539        0.5639   
                stepdown/other              0.9583        0.4944   

                                      Case 2 AUPRC Case 3 AUROC Case 3 AUPRC  \
Subpopulation   eICU Type                                                      
UnitAdmitSource Emergency Department        0.3158       0.9712       0.9449   
                Other ICU                   0.3467       0.9696       0.9332   
                Direct Admi

In [53]:
result.to_excel(excel_writer='external_validation_baseline_method.xlsx')